In [1]:
import pandas as pd
import re
from urllib import request
import bs4
import lxml

import requests
import numpy as np
import time

In [2]:
# connexion au drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# importer le dataframe avec le nom des pays
df_IQ_pays=pd.read_csv("/content/drive/MyDrive/Projets/GoogleMaps_scrapping/IQ_pays.csv")
df_pays=df_IQ_pays.loc[:,["country"]]
df_pays.head()

,country
0,India
1,China
2,United States
3,Indonesia
4,Pakistan


In [4]:
list_pays=list(df_pays.iloc[:,0])
list_pays=list(map(lambda pays:pays.replace('DR Congo','Congo'),list_pays))

## Collecte des codes html

In [ ]:
def save_html_GoogleMaps_Pays(list_pays, keyWord="museum"):

	for pays in list_pays :

		payload = {
			"source": "google",
			"url": f"https://www.google.com/search?q="+keyWord+"+"+pays+"&sca_esv=6f64e94da92d3a6a&authuser=2&hl=fr&tbm=lcl&sxsrf=ACQVn09J4OOLuuURzPgF8P-yeyrYHTqJ8A%3A1707770220629&ei=bIHKZZ2HJoOGkdUPk_yOsAM&ved=0ahUKEwid7e7q06aEAxUDQ6QEHRO-AzYQ4dUDCAk&uact=5&oq=museum+france&gs_lp=Eg1nd3Mtd2l6LWxvY2FsIg1tdXNldW0gZnJhbmNlMgUQABiABDIIEAAYgAQYywEyBhAAGBYYHjIIEAAYFhgeGA8yBhAAGBYYHjIGEAAYFhgeMgYQABgWGB4yBhAAGBYYHjIGEAAYFhgeMgYQABgWGB5I4TpQ6iVY-i1wAHgAkAEAmAGfAaABmgaqAQM0LjS4AQPIAQD4AQHCAgQQABgewgIGEAAYBRgewgIGEAAYCBgewgIIEAAYFhgeGArCAgoQABiABBgUGIcCwgIHEAAYgAQYCogGAQ&sclient=gws-wiz-local",
			"geo_location": pays,
		}

		response = requests.request(
		"POST",
		"https://realtime.oxylabs.io/v1/queries",
		auth=("BinomeDalasGM", "Binome2_racha_ahmed"),
		json=payload,
		timeout=180,
		)
		html = response.json().get("results")[0].get("content")
		np.savetxt("/content/drive/MyDrive/Projets/GoogleMaps_scrapping/"+pays+"_+"+keyWord+"s.txt", np.array([''.join(list(html))]), fmt='%s')
		time.sleep(5)


save_html_GoogleMaps_Pays(list_pays,keyWord="museum")

In [ ]:
save_html_GoogleMaps_Pays(list_pays,keyWord="bibliothèque")

In [ ]:
pays="Congo"
#keyWord="bibliothèque"
keyWord="museum"
payload = {
  "source": "google",
  "url": f"https://www.google.com/search?q="+keyWord+"+"+pays+"&sca_esv=6f64e94da92d3a6a&authuser=2&hl=fr&tbm=lcl&sxsrf=ACQVn09J4OOLuuURzPgF8P-yeyrYHTqJ8A%3A1707770220629&ei=bIHKZZ2HJoOGkdUPk_yOsAM&ved=0ahUKEwid7e7q06aEAxUDQ6QEHRO-AzYQ4dUDCAk&uact=5&oq=museum+france&gs_lp=Eg1nd3Mtd2l6LWxvY2FsIg1tdXNldW0gZnJhbmNlMgUQABiABDIIEAAYgAQYywEyBhAAGBYYHjIIEAAYFhgeGA8yBhAAGBYYHjIGEAAYFhgeMgYQABgWGB4yBhAAGBYYHjIGEAAYFhgeMgYQABgWGB5I4TpQ6iVY-i1wAHgAkAEAmAGfAaABmgaqAQM0LjS4AQPIAQD4AQHCAgQQABgewgIGEAAYBRgewgIGEAAYCBgewgIIEAAYFhgeGArCAgoQABiABBgUGIcCwgIHEAAYgAQYCogGAQ&sclient=gws-wiz-local",
  "geo_location": pays,
}

response = requests.request(
"POST",
"https://realtime.oxylabs.io/v1/queries",
auth=("BinomeDalasGM", "Binome2_racha_ahmed"),
json=payload,
timeout=180,
)
html = response.json().get("results")[0].get("content")
np.savetxt("/content/drive/MyDrive/Projets/GoogleMaps_scrapping/"+pays+"_+"+keyWord+"s.txt", np.array([''.join(list(html))]), fmt='%s')


## Création du dataset (scrapping musées)

In [5]:
# construire la liste des pays avec les données des musées
def get_df_pays_musees(list_pays):

  #dictionnaire de correspondance des chiffres bangla aux chiffres standards
  bangla_numerals ={
          '১': '1',
          '২': '2',
          '৩': '3',
          '৪': '4',
          '৫': '5',
          '৬': '6',
          '৭': '7',
          '৮': '8',
          '৯': '9',
          '০': '0'
  }
  def convert_bangla_number(num):
    #remplacer chaque chiffre bangla par son équivalent standards
    for bangla, standard in bangla_numerals.items():
        num = num.replace(bangla, standard)
    return num

  #dictionnaire de correspondance des chiffres arabes indiens aux chiffres standards
  arabic_numerals = {
        '٠': '0', '١': '1', '٢': '2', '٣': '3', '٤': '4',
        '٥': '5', '٦': '6', '٧': '7', '٨': '8', '٩': '9'
  }
  def convert_arabic_number(num):
    #remplacer chaque chiffre arabe indien par son équivalent standard
    for arabic, standard in arabic_numerals.items():
        num = num.replace(arabic, standard)

    #remplacer le point arabe par le point
    num = num.replace('٫', '.')

    return num

  res_pays_musees=[]
  for pays in list_pays :
    link_pays_musees=("/content/drive/MyDrive/Projets/GoogleMaps_scrapping/musees/"+pays+"_museums.txt")
    with open(link_pays_musees, "r") as f:
      html_pays_musees=f.readlines()

    html_pays_musees=''.join(html_pays_musees)
    soup= bs4.BeautifulSoup(html_pays_musees, "lxml")
    list_musees_brut=soup.find_all("div", {"class": "rllt__details"})
    notes_musees=[ ]

    for musee in list_musees_brut:
      if(musee.find("span", {'class': 'yi40Hd'}) !=None):
        note=musee.find("span", {'class': 'yi40Hd'}).text
        if (note[0] in list(range(0,10))):
          notes_musees.append(float(note.replace(',','.')) )
        elif(note[0] in arabic_numerals):
          notes_musees.append(float(convert_arabic_number(note)) )
        elif(note[0] in bangla_numerals):
          notes_musees.append(float(convert_bangla_number(note)) )
        else :
          notes_musees.append(note)

    #nom_musees=[musee.find("span").text for musee in list_musees_brut]
    nbVotes_musees=[musee.find("span", {'class': 'RDApEe'}).text.replace(',','.') for musee in list_musees_brut if musee.find("span", {'class': 'RDApEe'}) !=None]
    nbVotes_musees_clean=[]
    for nbVote in nbVotes_musees:
      nbVote_num=str(re.search(r'\d+',nbVote ).group())
      if(len(nbVote_num)+2==len(nbVote)): #le nombre de votants n'est PAS exprimé en milliers (k) --> ne contient que les '(' et ')' en plus de ses chiffres
        nbVotes_musees_clean.append(float(nbVote_num))
      else :
        nbVotes_musees_clean.append(float(nbVote_num)*1000)

    res_pays_musees.append([pays, notes_musees, nbVotes_musees_clean ])
    df_pays_musees=pd.DataFrame(res_pays_musees, columns=['Pays', "Notes_musees", "NbVotes_musees_clean" ])

  return df_pays_musees
df_pays_musees=get_df_pays_musees(list_pays)
df_pays_musees

,Pays,Notes_musees,NbVotes_musees_clean
0,India,"[4.6, 4.5, 4.4, 4.6, 4.6, 4.3, 4.5, 4.4, 4.5, ...","[18000.0, 49000.0, 69000.0, 34000.0, 74000.0, ..."
1,China,"[4.3, 4.4, 4.6, 4.5, 4.3, 4.4, 4.6, 4.6, 4.3, ...","[398.0, 925.0, 1000.0, 236.0, 154.0, 451.0, 38..."
2,United States,"[4.3, 4.5, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, ...","[373.0, 19000.0, 46000.0, 30000.0, 32000.0, 16..."
3,Indonesia,"[4,7, 4,5, 4,5, 4,6, 4,3, 4,7, 4,6, 4,7, 4,5, ...","[1000.0, 157000.0, 19.0, 22000.0, 205.0, 9000...."
4,Pakistan,"[4.3, 4.5, 4.5, 4.6, 4.4, 4.4, 4.6, 4.5, 4.8, ...","[1000.0, 4000.0, 8000.0, 13000.0, 12000.0, 400..."
...,...,...,...
192,Turks and Caicos Islands,"[4.4, 4.6, 5.0, 3.8, 4.6]","[187.0, 36.0, 4.0, 9.0, 7.0]"
193,Marshall Islands,[4.2],[34.0]
194,Liechtenstein,"[4.6, 4.3, 3.7, 4.3, 4.2, 4.3, 4.2, 4.4, 4.8, ...","[280.0, 645.0, 64.0, 260.0, 17.0, 22.0, 18.0, ..."
195,British Virgin Islands,"[4.7, 4.4, 5.0, 2.5, 3.8]","[26.0, 7.0, 1.0, 2.0, 5.0]"
